In [1]:
import mysql.connector as conn
user_name = 'root'
password = 'Newvision@143'
try:
    connection = conn.connect(username =user_name,password =password)
    print("Successfully Connected")
except conn.Error as e:
    print(e.errno)

Successfully Connected


In [2]:
cursor = connection.cursor()

In [3]:
##use databse 
cursor.execute("use Ourstore")

Task 1:

Little Lemon is running a marketing campaign this month. They need to issue a discount coupon to the top spender on daily bases. 

Create a stored procedure TopSpender that can retrieve the following:

the booking ID, 

guest’s full name, 

and the bill amount of the top spender at closing. 

Call the procedure and print the results.  

In [4]:
stored_procedure ="""
create procedure TopSpender() 
Begin 
select 
	b.BookingId,Concat(b.GuestFirstName,' ' ,b.GuestLastName ) as full_name
    ,o.BillAmount
from Bookings b inner join Orders o
on b.BookingId = o.BookingId 
Order by BillAmount desc limit 1; 
End
"""
cursor.execute(stored_procedure)

In [7]:
##Call the procedure
cursor.callproc("TopSpender")
##retrieve results 
results = next(cursor.stored_results())
data = results.fetchall()

for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]
print(columns)
for data in data:
    print(data)

['BookingId', 'full_name', 'BillAmount']
(1, 'Anna Iversen', 86)


Task 2:

Help Little Lemon to retrieve all those bookings where the guests did not appear today. How will you implement this task using a stored procedure? Use `NoArrival` as a name for your stored procedure.  

TIP: Target the `Orders` and the `Bookings` tables, join them on `BookingID` and retrieve the records with a `NULL` value in the bill amount. Create a stored procedure, call it by its name using python and print the results.

In [9]:
procedure_query="""
CREATE PROCEDURE NoArrival()

BEGIN

SELECT bookings.BookingID, 
Orders.BillAmount 
FROM Bookings
LEFT JOIN
Orders ON Bookings.BookingID=Orders.BookingID
WHERE BillAmount IS NULL;

END

"""
cursor.execute(procedure_query)


In [11]:
# Execute the query 


cursor.callproc("NoArrival")


results = next( cursor.stored_results() )
dataset = results.fetchall()

for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]


print(columns)


for data in dataset:
    print(data)

['BookingID', 'BillAmount']
(6, None)


Task 3:

It is very important for little lemon to keep track of the status of each guest’s order and display it on the screen to keep their guests informed.  

This is how they categorize the orders: 

* If not assigned to any employee, the status is `In Queue` 
* If assigned to the employees with IDs 4 and 5, the status is `Preparing Order` 
* If assigned to the employees with IDs 1, 2, and 3, the status is `Order Served` 

Create a stored procedure named `OrderStatus` for the above task and call to check if everything is working.  

In [12]:
stored_procedure_query="""
CREATE PROCEDURE OrderStatus()

BEGIN

SELECT bookingID, 
CASE
WHEN employeeID IN (1,2,3) THEN "Order Served" 
WHEN employeeID IN (4,5) THEN "Preparing Order" 
ELSE "In Queue" 
END AS Status
FROM Bookings;

END

"""
cursor.execute(stored_procedure_query)
cursor.callproc("OrderStatus")
results = next( cursor.stored_results() )
dataset = results.fetchall() 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]
print(columns)
for data in dataset:
    print(data)

['bookingID', 'Status']
(1, 'Order Served')
(2, 'Order Served')
(3, 'Order Served')
(4, 'Preparing Order')
(5, 'Order Served')
(6, 'Preparing Order')
